In [1]:
!pip install influxdb_client influxdb

In [2]:
pip install pyspark 

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datetime import datetime
from influxdb import DataFrameClient
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [4]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
spark = SparkSession.builder.appName("Test").getOrCreate()
df = pd.read_excel('SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0'])

df.head()
#sdf = spark.createDataFrame(df) sdf.show()

,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
0,2019-07-20 08:39:59.004013+00:00,4.323736,492.896881,2,2,2,1,131.408615,9.313829,257.933868,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
1,2019-07-20 08:39:58.005004800+00:00,4.323736,492.465100,2,2,2,1,131.408615,9.316713,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
2,2019-07-20 08:39:57.005004800+00:00,4.303558,492.308100,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
3,2019-07-20 08:39:56.005004800+00:00,4.253915,491.405273,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
4,2019-07-20 08:39:55.001007+00:00,4.200429,491.169769,2,2,2,1,131.408615,9.319918,257.703156,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


In [5]:
df['LS 201']=np.where(df['LS 201'] == 'Active', 1, 0)
df['LS 202']=np.where(df['LS 202'] == 'Active', 1, 0)
df['LSL 203']=np.where(df['LSL 203'] == 'Inactive', 0, 1)
df['LSLL 203']=np.where(df['LSLL 203'] == 'Active', 1, 0)
df['LS 401']=np.where(df['LS 401'] == 'Active', 1, 0)
df['LSH 601']=np.where(df['LSH 601'] == 'Active', 1, 0)
df['LSH 602']=np.where(df['LSH 602'] == 'Active', 1, 0)
df['LSH 603']=np.where(df['LSH 603'] == 'Active', 1, 0)
df['LSL 601']=np.where(df['LSL 601'] == 'Active', 1, 0)
df['LSL 602']=np.where(df['LSL 602'] == 'Active', 1, 0)
df['LSL 603']=np.where(df['LSL 603'] == 'Active', 1, 0)

In [6]:
#We first prepare the data for our experiments. Let us load a matrix from the file which has  14996  samples and  77  features.
# prepare data
import numpy as np
filepath ='SWaT_dataset_Jul 19 v2.xlsx'
matrixA =df.drop('GMT +0',axis='columns').T
print('The matrix A has ', matrixA.shape[0], ' rows and ', matrixA.shape[1], ' columns.')

The matrix A has  77  rows and  14996  columns.


In [7]:
#df.drop('GMT +0',axis='columns')

In [8]:
matrixA=df[['FIT 101','LIT 101','AIT 201','AIT 202']].T

In [9]:
matrixA

,0,1,2,3,4,5,6,7,8,9,...,14986,14987,14988,14989,14990,14991,14992,14993,14994,14995
FIT 101,4.323736,4.323736,4.303558,4.253915,4.200429,4.109150,3.955097,3.720975,3.433046,3.057682,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LIT 101,492.896881,492.465100,492.308100,491.405273,491.169769,491.169769,491.169769,491.287500,491.326782,491.405273,...,725.273200,725.783500,726.097534,726.490051,727.157349,727.706900,728.688200,729.120000,729.434000,729.865800
AIT 201,131.408615,131.408615,131.408615,131.408615,131.408615,131.408615,131.408615,131.408615,131.536789,131.536789,...,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557
AIT 202,9.313829,9.316713,9.317354,9.317354,9.319918,9.321199,9.321199,9.321199,9.321840,9.323122,...,9.289157,9.289157,9.289157,9.289157,9.289157,9.289157,9.289157,9.293002,9.293002,9.293002


In [10]:
matrixA

,0,1,2,3,4,5,6,7,8,9,...,14986,14987,14988,14989,14990,14991,14992,14993,14994,14995
FIT 101,4.323736,4.323736,4.303558,4.253915,4.200429,4.109150,3.955097,3.720975,3.433046,3.057682,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LIT 101,492.896881,492.465100,492.308100,491.405273,491.169769,491.169769,491.169769,491.287500,491.326782,491.405273,...,725.273200,725.783500,726.097534,726.490051,727.157349,727.706900,728.688200,729.120000,729.434000,729.865800
AIT 201,131.408615,131.408615,131.408615,131.408615,131.408615,131.408615,131.408615,131.408615,131.536789,131.536789,...,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557,142.527557
AIT 202,9.313829,9.316713,9.317354,9.317354,9.319918,9.321199,9.321199,9.321199,9.321840,9.323122,...,9.289157,9.289157,9.289157,9.289157,9.289157,9.289157,9.289157,9.293002,9.293002,9.293002


In [11]:
matrixA=matrixA.values

In [12]:
matrixA[:, 1]

array([  4.32373571, 492.4651    , 131.408615  ,   9.316713  ])

In [13]:
np.sum(matrixA[:, 1])

637.51416371


2. Algorithm Description
The beauty of the count sketch is that given  A∈Rm×n , a sketch of any size can be formed in  O(nnz(A))  time, where  nnz(A)  denotes the number of non-zero entries in  A .
Suppose we are given any fixed matrix  A∈Rm×n  and the size of sketch  s . We first hash each column with a discrete value which is uniformly sampled from  {1,⋯,s} , then flip the sign of each column with probability 50%, and finally sum up the columns with the same hash value. The result is an  m×s  matrix  C=AS .

The matrix  A  is normally too large to fit in memory, but the sketch  C  can fit in memory. In such situation, we can read  A  in a streaming fashion, that is, keep only one column of  A  in memory at a time. 

In [14]:
def countSketchStreaming(matrixA, s):
    m, n = matrixA.shape
    matrixC = np.zeros([m, s])
    hashedIndices = np.random.choice(s, n, replace=True)
    randSigns = np.random.choice(2, n, replace=True) * 2 - 1 
    for j in range(n):
        a = matrixA[:, j]
        h = hashedIndices[j]
        g = randSigns[j]
        matrixC[:, h] += g * a
    return matrixC

s = 2 # sketch size, can be tuned
matrixC = countSketchStreaming(matrixA, s)

In [15]:

# Test
# compare the l2 norm of each row of A and C
rowNormsA = np.sqrt(np.sum(np.square(matrixA), 1))
print(rowNormsA)
rowNormsC = np.sqrt(np.sum(np.square(matrixC), 1))
print(rowNormsC)

[  219.76851414 90900.6646424  17023.20833846  1128.04779413]
[   284.25384423 112567.19462251  20994.89515709   1424.07728062]


Import Data To InfluxDB

In [16]:
df = pd.DataFrame(matrixC , columns = ['Column_A','Column_B'])
sdf = spark.createDataFrame(df) 

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [17]:
sdf

DataFrame[Column_A: double, Column_B: double]

In [32]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

token = 'Uia8RRskYI4eznCuUeT1ushn7CNj1vZy867-Upx7-LEWcKiXAaatbqEZGZ2acmxfuDmDiAF460YCnV_Xjjw-9Q=='
org = 'changeme'
bucket = 'test'

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for row in dataCollect:
        for col in sdf.columns[1:]:
            p = influxdb_client.Point("swat_measure_2").field(col, row[col])
            write_api.write(bucket, org, p)  

In [33]:
with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for row in dataCollect:
        for col in sdf.columns[1:]:
            p = influxdb_client.Point("swat_measure_2").field(col, row[col])
            write_api.write(bucket, org, p)  

In [34]:
dataCollect = sdf.collect()
for row in dataCollect:
    for col in sdf.columns[1:]:
        p = influxdb_client.Point("swat_measure_2").field(col, row[col])
        print(p)

swat_measure_2 Column_B=-69.88981900999991
swat_measure_2 Column_B=42940.199887000024
swat_measure_2 Column_B=7878.684454999999
swat_measure_2 Column_B=485.3937313799988


In [35]:
p